<h2>Создание датасета</h2>

In [75]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [76]:
df = pd.read_csv('sale_tasks_dataset.csv', decimal='.')

In [77]:
df['finish_dttm'] = pd.to_datetime(df['finish_dttm'], format='%Y-%m-%d %H:%M:%S')
df['day_of_week'] = df['finish_dttm'].dt.dayofweek
df['is_holiday'] = df.day_of_week.apply(lambda x: 1 if x > 4 else 0) # создание is_holiday
df['finish_dttm'] = df['finish_dttm'].dt.strftime('%Y-%m-%d')

df['approval_flg'] = df['wo_hit_status_result_desc'].map({'Дозвон, Отказ': 0, 'Дозвон, Успешно': 1})

df['age'] = pd.to_numeric(df['age'], errors='coerce')

df["bundle_nm"] = df["bundle_nm"].fillna(0)
df["bundle_flg"] = df["bundle_nm"].map({'XXX': 0, 'Pro': 1, 'Premium': 1})

df['gender_cd'] = df['gender_cd'].map({'F': 0, 'M': 1})

df['region_size'].fillna('no', inplace=True)
mapping = {
    'No': 0,
    'town': 1,
    'urban': 2,
    'msk': 3,
    'rural': 4
}

df['region_size_numeric'] = df['region_size'].map(mapping)

In [78]:
df.to_csv('data_preprocessed.csv', index=False)

<h3>Чистка выбросов</h3>

In [79]:
df.dropna(subset=['age', 'gender_cd'], inplace=True) # пустые значения по age, gender_cd

In [80]:
df = df[(df['age'] <= 65) & (df['age'] >= 14)]

In [81]:
df.drop_duplicates(subset=['customer_id'], inplace=True, keep='first') # дубликаты клиентов
df.drop(df[df['customer_id'] == '538327981'].index, inplace=True)

In [82]:
IQR = df["state_talk_time_sec"].quantile(0.75) - df["state_talk_time_sec"].quantile(0.25)
high = df["state_talk_time_sec"].quantile(0.75) + IQR * 3
df = df[df["state_talk_time_sec"] < high] # чистка длительности звонка

In [83]:
df.to_csv('data_preprocessed_sort.csv', index=False)